<a href="https://colab.research.google.com/github/hkrsmk/beepboop-shopee-code-league-2020/blob/master/logistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle data

In [ ]:
!mkdir .kaggle

In [ ]:
import json
token = {"username":[redacted],"key":[redacted]}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c logistics-shopee-code-league

  0% 0.00/9.01k [00:00<?, ?B/s]
100% 9.01k/9.01k [00:00<00:00, 15.4MB/s]
 96% 366M/381M [00:03<00:00, 130MB/s]
100% 381M/381M [00:03<00:00, 121MB/s]


In [ ]:
!unzip delivery_orders_march.csv.zip
!rm *.zip

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

# actual analysis

In [57]:
import pandas as pd
import datetime
import numpy as np

In [ ]:
dataset = pd.read_csv('delivery_orders_march.csv')

In [ ]:
dataset.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [ ]:
def convert(column):
  converted = pd.to_datetime(dataset[column] + 28800, unit = 's')
  return converted

convert('pick')

0         2020-03-02 16:39:57
1         2020-03-04 16:19:28
2         2020-03-04 15:20:34
3         2020-03-05 22:36:56
4         2020-03-04 18:38:25
                  ...        
3176308   2020-04-02 18:02:08
3176309   2020-04-03 14:23:31
3176310   2020-04-02 19:29:11
3176311   2020-04-03 13:19:54
3176312   2020-04-02 18:38:43
Name: pick, Length: 3176313, dtype: datetime64[ns]

In [ ]:
def datechange(array):
  for i in range(len(array)):
    dataset[array[i]] = convert(array[i])

  return dataset

datechange(['1st_deliver_attempt', '2nd_deliver_attempt'])

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02 18:02:08,2020-04-04 13:22:22,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,2020-04-03 14:23:31,2020-04-06 21:17:27,2020-04-07 11:52:11,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,2020-04-02 19:29:11,2020-04-04 09:55:40,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,2020-04-03 13:19:54,2020-04-07 11:19:12,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


In [ ]:
new_dataset = dataset

def droptime(array):
  for i in range(len(array)):
    new_dataset[array[i]] = dataset[array[i]].dt.date

  return new_dataset

droptime(['pick','1st_deliver_attempt', '2nd_deliver_attempt'])

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,2020-04-02,2020-04-04,NaT,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,2020-04-03,2020-04-07,NaT,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


In [62]:
# Working Days are defined as Mon - Sat, Excluding Public Holidays.
# SLA calculation begins from the next day after pickup (Day 0 = Day of Pickup; Day 1 = Next Day after Pickup)
# 2nd Attempt must be no later than 3 working days after the 1st Attempt,
# regardless of origin to destination route (Day 0 = Day of 1st Attempt; Day 1 = Next Day after 1st Attempt).

# if Sunday or these PHs, then don't count as working day
holidays = ['2020-03-25',
            '2020-03-30',
            '2020-03-31']

locations = ['Metro Manila',
             'Luzon',
             'Visayas',
             'Mindanao']

locations_s = ['metro manila',
             'luzon',
             'visayas',
             'mindanao']

slas = {'Metro Manila': [3, 5, 7, 7],
        'Luzon': [5, 5, 7, 7],
        'Visayas': [7, 7, 7, 7],
        'Mindanao': [7, 7, 7, 7]}

sla_data = pd.DataFrame(slas, columns = locations, index = locations)

In [109]:
sla_data = sla_data.astype('timedelta64[ns]')
sla_data

,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,3 days,5 days,7 days,7 days
Luzon,5 days,5 days,7 days,7 days
Visayas,7 days,7 days,7 days,7 days
Mindanao,7 days,7 days,7 days,7 days


In [79]:
sla_data.at['Luzon', 'Visayas']

7

In [64]:
dataset_locs = new_dataset

def find_locs(array, locs, loc2):
  for i in range(len(array)):
    for j in range(len(locs)):
      dataset_locs[array[i]] = np.where(dataset_locs[array[i]].apply(lambda x: (locs[j] in x) or (loc2[j] in x)),
                                        locs[j],
                                        dataset_locs[array[i]])
  return dataset_locs

find_locs(['buyeraddress', 'selleraddress'], locations, locations_s)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,Metro Manila,Metro Manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila
2,2220979489,2020-03-04,2020-03-06,NaT,Metro Manila,Metro Manila
3,2221066352,2020-03-05,2020-03-07,NaT,Metro Manila,Metro Manila
4,2222478803,2020-03-04,2020-03-06,NaT,Luzon,Metro Manila
...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,Metro Manila,Metro Manila
3176310,31504462290482,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila
3176311,31504851495943,2020-04-03,2020-04-07,NaT,Luzon,Metro Manila


In [68]:
dataset_locs['1st_diff'] = dataset_locs['1st_deliver_attempt'] - dataset_locs['pick']
dataset_locs['2nd_diff'] = dataset_locs['2nd_deliver_attempt'] - dataset_locs['1st_deliver_attempt']
dataset_locs.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,diff,1st_diff,2nd_diff
0,2215676524,2020-03-02,2020-03-05,NaT,Metro Manila,Metro Manila,3 days,3 days,NaT
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila,2 days,2 days,4 days
2,2220979489,2020-03-04,2020-03-06,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT
3,2221066352,2020-03-05,2020-03-07,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT
4,2222478803,2020-03-04,2020-03-06,NaT,Luzon,Metro Manila,2 days,2 days,NaT


In [69]:
dataset_locs.drop('diff', axis = 1)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_diff,2nd_diff
0,2215676524,2020-03-02,2020-03-05,NaT,Metro Manila,Metro Manila,3 days,NaT
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila,2 days,4 days
2,2220979489,2020-03-04,2020-03-06,NaT,Metro Manila,Metro Manila,2 days,NaT
3,2221066352,2020-03-05,2020-03-07,NaT,Metro Manila,Metro Manila,2 days,NaT
4,2222478803,2020-03-04,2020-03-06,NaT,Luzon,Metro Manila,2 days,NaT
...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila,2 days,NaT
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,Metro Manila,Metro Manila,3 days,1 days
3176310,31504462290482,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila,2 days,NaT
3176311,31504851495943,2020-04-03,2020-04-07,NaT,Luzon,Metro Manila,4 days,NaT


In [113]:
# if contains sunday/ph, -1 day to diff
def check_exceptions():
  if all:
    exceptions_no += 1

  return exceptions_no

dataset_locs.at[0, '2nd_diff'] == NaT

NameError: ignored

In [129]:
# check buyeradd and selleradd for SLA
# account for second delivery

for i in range(1000000):
  if sla_data.at[dataset_locs.at[i, 'selleraddress'], dataset_locs.at[i, 'buyeraddress']] < dataset_locs.at[i, '1st_diff']:
    dataset_locs.at[i, 'is_late'] = 1
    
  else:
    dataset_locs.at[i,'is_late'] = 0

dataset_locs

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,diff,1st_diff,2nd_diff,is_late
0,2215676524,2020-03-02,2020-03-05,NaT,Metro Manila,Metro Manila,3 days,3 days,NaT,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Metro Manila,Metro Manila,2 days,2 days,4 days,0
2,2220979489,2020-03-04,2020-03-06,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT,0
3,2221066352,2020-03-05,2020-03-07,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT,0
4,2222478803,2020-03-04,2020-03-06,NaT,Luzon,Metro Manila,2 days,2 days,NaT,0
...,...,...,...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT,1
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,Metro Manila,Metro Manila,3 days,3 days,1 days,1
3176310,31504462290482,2020-04-02,2020-04-04,NaT,Metro Manila,Metro Manila,2 days,2 days,NaT,1
3176311,31504851495943,2020-04-03,2020-04-07,NaT,Luzon,Metro Manila,4 days,4 days,NaT,1


In [126]:
output = pd.DataFrame({'orderid': dataset_locs['orderid'],
                   'is_late': dataset_locs['is_late']})

output.fillna(0)
output.to_csv('outputz.csv', index=False)

In [127]:
output.count

<bound method DataFrame.count of                 orderid  is_late
0            2215676524        0
1            2219624609        0
2            2220979489        0
3            2221066352        0
4            2222478803        0
...                 ...      ...
3176308  31504087640510        1
3176309  31504147352227        1
3176310  31504462290482        1
3176311  31504851495943        1
3176312  31505199116340        1

[3176313 rows x 2 columns]>

In [ ]:
outputcsv = pd.read_csv('output.csv')
outputcsv.count

<bound method DataFrame.count of                 orderid  is_late
0            2215676524        0
1            2219624609        0
2            2220979489        0
3            2221066352        0
4            2222478803        0
...                 ...      ...
3176308  31504087640510        0
3176309  31504147352227        0
3176310  31504462290482        0
3176311  31504851495943        0
3176312  31505199116340        0

[3176313 rows x 2 columns]>

In [ ]:
!kaggle competitions submit -c logistics-shopee-code-league -f output.csv -m "beepboop submission"